**시나리오와 무관 공통 부분**

1. raw_data(.pdf, .hwp파일)을 전체 pdf로 변경
2. 청크된 문서 로드  
    2-1. chunk.pkl 파일이 없다면? -> pdf data로 부터 생성
    2-2. chunk.pkl 파일이 있다면? -> 바로 Load
3. 정답 스팬 생성  
    3-1. 2.의 청크된 문서로 부터 생성된 정답 스팬이 있다면? -> 생략  
    3-1. 2.의 청크된 문서로 부터 생성된 정답 스팬이 없다면? -> 생성  

**다음 부터는 시나리오에 따라 나누어짐**

4. 임베딩 모델을 선정하여 2.의 청크를 임베딩 및 벡터스토어 생성  
    - 임베딩 데이터를 저장해야하는가? -> 사실 faiss_index.idx만 있어도 속도가 나오는데  

===[실험 준비 끝]===  

5. 질문 입력   
    5.1. 준비한 질문이 있다면? -> 직접 질문 입력  
    5.2. 준비한 질문이 없다면? -> 공통된 질문을 자동으로 입력
6. 질문과 답변, 참고한 문서 등을 기록
7. 6.을 바탕으로 평가
8. 7.의 평가를 저장



# 기본 설정

## import

In [ ]:
from utils import experiment
import os
import json

## OpenAI API KEY

In [2]:
# 셀 내용 지우시고 각자의 방식으로 API KEY 설정하시면 됩니다.
from APIkey_loader import OpenAi_API_KEY

os.environ["OPENAI_API_KEY"] = OpenAi_API_KEY

## 변수 설정

In [3]:
experiment_name="test20250804"
version="2.0"

# 실험

In [4]:
experiment.experiment(experiment_name=experiment_name, version=version)

[1] 실험 구조 설정
실험 설정 파일을 불러옵니다.
실험 설정을 불러왔습니다.: {'version': '2.0', 'test_name': 'test20250804', 'embedding_model': 'text-embedding-3-small', 'llm_model': 'gpt-4.1-mini', 'chunk_size': 100, 'chunk_overlap': 10, 'top_k': 10}
README.md 파일이 존재합니다. 실험 내용을 기록해주세요.

[2] Document Chunking
청크 파일이 존재합니다.
청크 문서를 로드 완료하였습니다.
문서 개수: 139494

[3] 정답 스팬 생성
정답 스팬이 ./data/processed/span//span_list_from_chunksize_100_overlap10.json에 저장되었습니다.
정답 스팬을 생성하였습니다.

[4] 임베딩 및 리트리버 생성
임베딩을 설정하였습니다. 
모델: text-embedding-3-small


c:\Users\User\Sprint\Project\Project_2\codeit_mid_project\utils\experiment.py:266: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model=embedding_model_name)
c:\Users\User\Sprint\Project\Project_2\codeit_mid_project\rag_chain\RagChain_temp.py:25: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(model_name=model_name, temperature

FAISS 인덱스를 불러왔습니다
retriever를 생성하였습니다. Top K: 10

[5] RAG Chain
RAG Chain을 생성했습니다. llm 모델: gpt-4.1-mini

[6] 질의 테스트를 시작합니다
질의 테스트를 종료합니다.
응답이 저장되었습니다.


# 평가

In [5]:
config_path = f"experiment/{experiment_name}/experiment_config.json"

with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
embedding_model_name = config.get("embedding_model")
llm_model_name = config.get("llm_model")
chunk_size = config.get("chunk_size")
chunk_overlap = config.get("chunk_overlap")
top_k = config.get("top_k")

In [ ]:
# 리트리버 평가
from utils import evaluate

# 파일 경로 지정
retrieved_path = f"experiment/{experiment_name}/result.jsonl"
gold_path = f"data/processed/span/span_list_from_chunksize_{chunk_size}_overlap{chunk_overlap}.json"

# 데이터 불러오기
retrieved_dict = evaluate.load_jsonl(retrieved_path)
gold_dict = evaluate.load_gold(gold_path)

# 점수 계산 및 출력
evaluate.evaluate_recall_at_k(retrieved_dict, gold_dict, k=3)
evaluate.evaluate_ndcg_at_k(retrieved_dict, gold_dict, k=3)
# evaluate.f1_score_at_k(retrieved_dict, gold_dict, k=3)